# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


## Import Libraries and Define Auxiliary Functions


In [1]:
import requests
import pandas as pd
import numpy as np
import datetime

# # Setting this option will print all collumns of a dataframe
# pd.set_option('display.max_columns', None)
# # Setting this option will print all of the data in a feature
# pd.set_option('display.max_colwidth', None)

### Functions

In [2]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])
        
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])
         
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])
        
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [3]:
spacex_url="https://api.spacexdata.com/v4/launches/past"
response = requests.get(spacex_url)
print(response.content)

b'[{"fairings":{"reused":false,"recovery_attempt":false,"recovered":false,"ships":[]},"links":{"patch":{"small":"https://images2.imgbox.com/94/f2/NN6Ph45r_o.png","large":"https://images2.imgbox.com/5b/02/QcxHUb5V_o.png"},"reddit":{"campaign":null,"launch":null,"media":null,"recovery":null},"flickr":{"small":[],"original":[]},"presskit":null,"webcast":"https://www.youtube.com/watch?v=0a_00nJ_Y88","youtube_id":"0a_00nJ_Y88","article":"https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html","wikipedia":"https://en.wikipedia.org/wiki/DemoSat"},"static_fire_date_utc":"2006-03-17T00:00:00.000Z","static_fire_date_unix":1142553600,"net":false,"window":0,"rocket":"5e9d0d95eda69955f709d1eb","success":false,"failures":[{"time":33,"altitude":null,"reason":"merlin engine failure"}],"details":"Engine failure at 33 seconds and loss of vehicle","crew":[],"ships":[],"capsules":[],"payloads":["5eb0e4b5b6c3bb0006eeb1e1"],"launchpad":"5e9e4502f5090995de566f86","flight_number":1,"name

### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [4]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [5]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [6]:
data = pd.json_normalize(response.json())
data.head()

static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                      None                    NaN  False     0.0   

                     rocket success  \
0  5e9d0d95eda69955f709d1eb   False   
1  5e9d0d95eda69955f709d1eb   False   
2  5e9d0d95eda69955f709d1eb   False   
3  5e9d0d95eda69955f709d1eb    True   
4  5e9d0d95eda69955f709d1eb    True   

                                            failures  \
0  [{'time': 33, 'altitude': None, 'reason': 'mer...   
1  [{'time': 301, 'altitude': 289, 'reason': 'har...   
2  [{'time': 140, 'altitude': 35, 'reason': 'resi...   
3                                                 []   
4                                                 []   

                                             details crew ships  ...  \
0   Engine failure at 33 seconds and loss of vehicle   []    []  ...   
1  Successful first stage burn and transition to ...   []    []  ...   
2  Residual stage 1 thrust led to collision betwe...   []    []  ...   
3  Ratsat was carried to orbit on the first succe...   []    []  ...   
4                                               None   []    []  ...   

  links.reddit.media links.reddit.recovery links.flickr.small  \
0               None                  None                 []   
1               None                  None                 []   
2               None                  None                 []   
3               None                  None                 []   
4               None                  None                 []   

   links.flickr.original                                     links.presskit  \
0                     []                                               None   
1                     []                                               None   
2                     []                                               None   
3                     []                                               None   
4                     []  http://www.spacex.com/press/2012/12/19/spacexs...   

                                 links.webcast  links.youtube_id  \
0  https://www.youtube.com/watch?v=0a_00nJ_Y88       0a_00nJ_Y88   
1  https://www.youtube.com/watch?v=Lk4zQ2wP-Nc       Lk4zQ2wP-Nc   
2  https://www.youtube.com/watch?v=v0w9p3U8860       v0w9p3U8860   
3  https://www.youtube.com/watch?v=dLQ2tZEH6G0       dLQ2tZEH6G0   
4  https://www.youtube.com/watch?v=yTaIDooc8Og       yTaIDooc8Og   

                                       links.article  \
0  https://www.space.com/2196-spacex-inaugural-fa...   
1  https://www.space.com/3590-spacex-falcon-1-roc...   
2  http://www.spacex.com/news/2013/02/11/falcon-1...   
3               https://en.wikipedia.org/wiki/Ratsat   
4  http://www.spacex.com/news/2013/02/12/falcon-1...   

                                     links.wikipedia  fairings  
0              https://en.wikipedia.org/wiki/DemoSat       NaN  
1              https://en.wikipedia.org/wiki/DemoSat       NaN  
2  https://en.wikipedia.org/wiki/Trailblazer_(sat...       NaN  
3               https://en.wikipedia.org/wiki/Ratsat       NaN  
4             https://en.wikipedia.org/wiki/RazakSAT       NaN  

[5 rows x 43 columns]

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [9]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [10]:
# Uncomment if running for the first time
# ------------------------------------------
# Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

# Call getBoosterVersion
getBoosterVersion(data)
print('Got booster versions')

# Call getLaunchSite
getLaunchSite(data)
print('Got launch Sites')

# Call getPayloadData
getPayloadData(data)
print('Got payload data')

# Call getCoreData
getCoreData(data)
print('Got core data')
print('Done!')

# create dictionary
launch_dict = {'FlightNumber': list(data['flight_number']),
               'Date': list(data['date']),
               'BoosterVersion':BoosterVersion,
               'PayloadMass':PayloadMass,'Orbit':Orbit,
               'LaunchSite':LaunchSite,'Outcome':Outcome,
               'Flights':Flights,'GridFins':GridFins,
               'Reused':Reused,'Legs':Legs,
               'LandingPad':LandingPad,'Block':Block,
               'ReusedCount':ReusedCount,'Serial':Serial,
               'Longitude': Longitude,'Latitude': Latitude}

# create data frame
launch_data = pd.DataFrame(launch_dict)
launch_data.to_csv('launch_data.csv')


In [14]:
launch_data.head()

Unnamed: 0  FlightNumber        Date BoosterVersion  PayloadMass Orbit  \
0           0             1  2006-03-24       Falcon 1         20.0   LEO   
1           1             2  2007-03-21       Falcon 1          NaN   LEO   
2           2             4  2008-09-28       Falcon 1        165.0   LEO   
3           3             5  2009-07-13       Falcon 1        200.0   LEO   
4           4             6  2010-06-04       Falcon 9          NaN   LEO   

        LaunchSite    Outcome  Flights  GridFins  Reused   Legs LandingPad  \
0  Kwajalein Atoll  None None        1     False   False  False        NaN   
1  Kwajalein Atoll  None None        1     False   False  False        NaN   
2  Kwajalein Atoll  None None        1     False   False  False        NaN   
3  Kwajalein Atoll  None None        1     False   False  False        NaN   
4     CCSFS SLC 40  None None        1     False   False  False        NaN   

   Block  ReusedCount    Serial   Longitude   Latitude  
0    NaN            0  Merlin1A  167.743129   9.047721  
1    NaN            0  Merlin2A  167.743129   9.047721  
2    NaN            0  Merlin2C  167.743129   9.047721  
3    NaN            0  Merlin3C  167.743129   9.047721  
4    1.0            0     B0003  -80.577366  28.561857

### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [35]:
data_falcon9 = launch_data.query('BoosterVersion == "Falcon 9"')

Now that we have removed some values we should reset the FlgihtNumber column


In [36]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9.head()

Unnamed: 0  FlightNumber        Date BoosterVersion  PayloadMass Orbit  \
4           4             1  2010-06-04       Falcon 9          NaN   LEO   
5           5             2  2012-05-22       Falcon 9        525.0   LEO   
6           6             3  2013-03-01       Falcon 9        677.0   ISS   
7           7             4  2013-09-29       Falcon 9        500.0    PO   
8           8             5  2013-12-03       Falcon 9       3170.0   GTO   

     LaunchSite      Outcome  Flights  GridFins  Reused   Legs LandingPad  \
4  CCSFS SLC 40    None None        1     False   False  False        NaN   
5  CCSFS SLC 40    None None        1     False   False  False        NaN   
6  CCSFS SLC 40    None None        1     False   False  False        NaN   
7   VAFB SLC 4E  False Ocean        1     False   False  False        NaN   
8  CCSFS SLC 40    None None        1     False   False  False        NaN   

   Block  ReusedCount Serial   Longitude   Latitude  
4    1.0            0  B0003  -80.577366  28.561857  
5    1.0            0  B0005  -80.577366  28.561857  
6    1.0            0  B0007  -80.577366  28.561857  
7    1.0            0  B1003 -120.610829  34.632093  
8    1.0            0  B1004  -80.577366  28.561857

## Data Wrangling


In [37]:
data_falcon9.isnull().sum()

Unnamed: 0         0
FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        5
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [38]:
data_falcon9['PayloadMass'] = (data_falcon9['PayloadMass']
    .replace(np.nan, np.mean(data_falcon9['PayloadMass'])))

C:\Users\alexi\AppData\Local\Temp\ipykernel_53328\2473825157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9['PayloadMass'] = (


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


In [41]:
data_falcon9.to_csv('falcon9_data_1.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|


Copyright © 2021 IBM Corporation. All rights reserved.
